In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
import datetime
import json
import csv
from tqdm.notebook import tqdm
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions
import re

/Users/hoover/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# 填入供應商
# 範例: 真愛密碼 -> JCode
vendor = '2Sweet'

In [3]:
# 填入"商品系列"網址第一頁
URL = "https://www.2sweet.com.tw/v2/official/SalePageCategory/509373?sortMode=Curator"

In [4]:
# 填入“商品系列”縮寫以便命名
#範例: 2024龍年 -> DRG2024
type = 'DRG2024'

In [5]:
filename = vendor + type + '.csv'

In [6]:
# 以下為csv表格中必填資料(不常改動)

# 自訂"Product type"
sku = ''
# 是否上架Shopify賣場: "TRUE" 或 "FALSE"
published = "TRUE"
# 是否有子類別(尺寸、顏色、材質等等)，通常設為"無"
o1n = "Title"
o1v = "Default Title"
o2n, o2v, o3n, o3v = '', '', '', ''
# 重量 Variant Grams
vargram = 0
# Variant Inventory Tracker, Variant Inventory Qty, Variant Inventory Policy, Variant Fulfillment Service
vit, viq, vip, vfs = 'shopify', 1, 'deny', 'manual'
# 價格 Variant Price, Variant Compare At Price
v_price = 0
vcap = 0
# 運送 Variant Requires Shipping
vrship = 'TRUE'
# 稅 Variant Taxable
vtax = 'TRUE'
# 條碼 Variant Barcode
vbar = ''
# 圖片地址,順序 Image Src, Image Position, Image Alt Text
imgalt = ''
# 禮品卡 Gift Card
giftcard = 'FALSE'
# SEO Title, SEO Description
SEOtitle, SEOdis = '', ''
# Google Shopping / Google Product Category, Google Shopping / Gender,
# Google Shopping / Age Group, Google Shopping / MPN, Google Shopping / Condition,
# Google Shopping / Custom Product, Google Shopping / Custom Label 0,
# Google Shopping / Custom Label 1, Google Shopping / Custom Label 2,
# Google Shopping / Custom Label 3, Google Shopping / Custom Label 4
g1, g2 , g3, g4, g5, g6, g7, g8, g9, g10, g11 = '', '', '', '', '', '', '', '', '', '', ''
# Variant Image, Variant Weight Unit, Variant Tax Code, Cost per item
vimg, vwu, vtc, cpi = '', '', '', ''
# Included / 台灣, Price / 台灣, Compare At Price / 台灣, 
# Included / 國際, Price / 國際, Compare At Price / 國際
TWinc, TWp, TWcap, INTinc, INTp, INTcap = 'TRUE', '', '', 'TRUE', '', ''
# Status
status = 'archived'

In [7]:
csv_header = ['Handle', 'Title', 'Body (HTML)', 'Vendor', 'Product Category',
               'Type', 'Tags', 'Published', 'Option1 Name', 'Option1 Value', 
               'Option2 Name', 'Option2 Value', 'Option3 Name', 'Option3 Value', 
               'Variant SKU', 'Variant Grams', 'Variant Inventory Tracker', 
               'Variant Inventory Qty', 'Variant Inventory Policy',  
               'Variant Fulfillment Service', 'Variant Price', 
               'Variant Compare At Price', 'Variant Requires Shipping', 'Variant Taxable', 
               'Variant Barcode', 'Image Src', 'Image Position', 'Image Alt Text', 
               'Gift Card', 'SEO Title', 'SEO Description', 
               'Google Shopping / Google Product Category', 
               'Google Shopping / Gender', 'Google Shopping / Age Group', 
               'Google Shopping / MPN', 'Google Shopping / Condition', 
               'Google Shopping / Custom Product', 'Google Shopping / Custom Label 0', 
               'Google Shopping / Custom Label 1', 'Google Shopping / Custom Label 2', 
               'Google Shopping / Custom Label 3', 'Google Shopping / Custom Label 4',
               'Variant Image', 'Variant Weight Unit', 'Variant Tax Code', 
               'Cost per item', 'Included / 台灣', 'Price / 台灣', 'Compare At Price / 台灣', 
               'Included / 國際', 'Price / 國際', 'Compare At Price / 國際', 'Status']
df = pd.DataFrame(columns=csv_header)

In [8]:
# 找出"系列"頁面所有產品的網址
options = ChromeOptions()

options.add_argument("--headless=new")

driver = webdriver.Chrome(options=options)

driver.get(URL)

soup1 = BeautifulSoup(driver.page_source, 'lxml')

soup2 = soup1.find_all('a', class_='sc-bBqzUu')

url_list = []

for index, data in enumerate(soup2):
    url_list.append('https://www.2sweet.com.tw' + soup2[index].get('href'))

driver.quit()

In [9]:
# 找出所有產品的資料
for url in tqdm(url_list):

    #headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}

    #page = requests.get(url, headers=headers)

    #soup1 = BeautifulSoup(page.content, 'html.parser')

    #soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
    options = ChromeOptions()

    options.add_argument("--headless=new")

    driver = webdriver.Chrome(options=options)

    driver.get(url)

    soup1 = BeautifulSoup(driver.page_source, 'html.parser')

    soup2 = soup1.find('script', string=re.compile(r'window.ServerRenderData\["SalePageIndexViewModel"\]\s*=\s*({.*?});'))

    pattern = r'window\.ServerRenderData\["SalePageIndexViewModel"\]\s*=\s*({.*?});'
    match = re.search(pattern, soup2.text)
    scrap1 = json.loads(match.group(1))

    # 抓取需要的商品的文字資訊
    title = scrap1.get('Title')
    description = scrap1.get('SubDescript')
    if description == '':                               #有些資料放在ShortDescription
        description = scrap1.get('ShortDescription')
    variant = scrap1.get("MajorList")[0]['SKUList'][0]['PropertyList']
    var = [i['Name'] for i in variant]
    if var == ['-1']:                              #有些產品沒有種類
        var = []

    # 抓取商品圖片網址
    image_count = scrap1.get('ImageCount')
    image = []
    for i in range (0, image_count):
        image_url = scrap1.get('ImageList')[i]['PicUrl']
        image.append('https:' + image_url)

    # 自動歸類"Product Category"
    prod_cat = ""
    if '手鍊' in description or '手鍊' in title:
        prod_cat = "Apparel & Accessories > Jewelry > Bracelets"
    if '戒' in description or '戒' in title:
        prod_cat = "Apparel & Accessories > Jewelry > Rings"
    if '項鍊' in description or '項鍊' in title:
        prod_cat = "Apparel & Accessories > Jewelry > Necklaces"
    if '耳' in description or '耳' in title:
        prod_cat = "Apparel & Accessories > Jewelry > Earrings"
    if '墜飾' in description  or '墜子' in description or '墜' in description or '墜飾' in title  or '墜子' in title or '墜' in title:
        prod_cat = "Apparel & Accessories > Jewelry > Charms & Pendants"
    if '擺件' in description or '擺飾' in description or '禮盒' in description or '擺件' in title or '擺飾' in title or '禮盒' in title:
        prod_cat = "Apparel & Accessories > Jewelry > Jewelry Sets"

    # 自動歸類“Tags”
    tag = []
    if '金' in description:
        tag.append('黃金')
    if '銀' in description:
        tag.append('銀')
    if '手鍊' in description or '手鍊' in title:
        tag.append('手鍊')
    if '戒' in description or '戒' in title:
        tag.append('戒指')
    if '項鍊' in description or '項鍊' in title:
        tag.append('項鍊')
    if '耳' in description or '耳' in title:
        tag.append('耳環')
    if '墜飾' in description  or '墜子' in description or '墜' in description or '墜飾' in title  or '墜子' in title or '墜' in title:
        tag.append('墜子')
    if '擺件' in description or '擺飾' in description or '禮盒' in description or '擺件' in title or '擺飾' in title or '禮盒' in title:
        tag.append('擺件')
    tag = ", ".join(tag)

    # 將資料寫入表格中
    data = []
    for index, imgurl in enumerate(image):
        if index == 0:                      # For the first line of every product, lines corrresponds to number of images, does not consider if variant > images
            try:                            # If var has data, o1v_ = var[index], o1n_ = 'Style', other variables same
                o1v_ = var[index]
                o1n_ = 'Style'
            except:                         # If var is [], var[index] error, go to except, use default setting.
                o1v_, o1n_ = o1v, o1n
            data = [[title, title, description, vendor, prod_cat, type, tag, published, o1n_, o1v_, 
                o2n, o2v, o3n, o3v, sku, vargram, vit, viq, vip, vfs, v_price, vcap, vrship, vtax, 
                vbar, imgurl, index+1, imgalt, giftcard, SEOtitle, SEOdis, g1 , g2, g3, g4, g5, 
                g6, g7, g8, g9, g10, g11, vimg, vwu, vtc, cpi, TWinc, TWp, TWcap, INTinc, INTp, 
                INTcap, status]]
        else:                               # For second line and later for every product
            try:
                o1v_ = var[index]
                vargram_, vit_, viq_, vip_, vfs_, v_price_, vrship_, vtax_, vwu_ = vargram, vit, viq, vip, vfs, v_price, vrship, vtax, vwu
            except:
                o1v_, vargram_, vit_, viq_, vip_, vfs_, v_price_, vrship_, vtax_, vwu_ = '', '', '', '', '', '', '', '', '', ''
            data.append([title, '', '', '', '', '', '', '', '', o1v_, '', '', '', '', '', vargram_, 
                         vit_, viq_, vip_, vfs_, v_price_, '', vrship_, vtax_, '', imgurl, index+1, 
                         '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', vwu_, '', 
                         '', '', '', '', '', '', '', ''])
    
    new_df = pd.DataFrame(data, columns=csv_header)
    df = pd.concat([df, new_df], ignore_index=True)

    # 設置延遲時間
    time.sleep(0.3)

with open(filename, 'w', newline='', encoding='UTF8') as file:
    df.to_csv(file, index=False)

  0%|          | 0/36 [00:00<?, ?it/s]